Best submission so far is ensemble with LinearSVC (not SVM), no max features, no KNN. (KNN hasn't been grid searched yet).

In [50]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [51]:
%matplotlib

Using matplotlib backend: MacOSX


In [52]:

import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np
%matplotlib inline
sns.set_palette('colorblind')
sns.set_style('white')

In [99]:
import pandas as pd
import nltk
import string

from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC, SVC
from sklearn.cross_validation import StratifiedShuffleSplit, train_test_split
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix 

In [54]:
from nltk.corpus import brown

In [55]:
from collections import Counter

In [56]:
from random import shuffle

In [57]:
def get_data():
    df = pd.read_csv('newtrain.csv')
    return df

In [58]:
def get_test_data():
    df = pd.read_csv('newtest.csv')
    return df

In [59]:
def add_features(df):

    #tokenize text
    pattern = r'''(?x)    # set flag to allow verbose regexps
         ([A-Z]\.)+        # abbreviations, e.g. U.S.A.
       | \w+([-']\w+)*        # words with optional internal hyphens
       | \$?\d+(\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
       | [!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+  # these are separate tokens (string.punctuation)
     '''
    tokenize = lambda text: nltk.regexp_tokenize(text, pattern)
    df['Tokens'] = df['Text'].apply(tokenize)
    
    def pos_tag(text):
        tuples = nltk.pos_tag(text)
        tags = []
        for t in tuples:
            tags.append(t[1])
        return tags
    
    def get_nouns(text):
        wnlemmatizer = nltk.WordNetLemmatizer()
        tuples = nltk.pos_tag(text)
        tags = []
        for t in tuples:
            if t[1][0] == 'N':
                tags.append(wnlemmatizer.lemmatize(t[0]))
        return tags
    df['Nouns'] = df['Tokens'].apply(get_nouns)
    df['POS'] = df['Tokens'].apply(pos_tag)
    make_string = lambda a: ' '.join(i for i in a)
    df['Nouns'] = df['Nouns'].apply(make_string)
    df['POS'] = df['POS'].apply(make_string)
    
    df['TokenString'] = df['Tokens'].apply(make_string)
    
    df['TokensNouns'] = df[['TokenString', 'Nouns']].apply(lambda x: ' '.join(x), axis=1)
    
    return df

In [102]:
#vectorize data
#turns words into a list of vectors - vector length is the total number of words
#Vector elements correspond to 1 word (1/0 if word is/not present in the current item)
def featurize(df, test_df, count_vec):
    #count_vec = CountVectorizer(max_features=max_f, analyzer = 'char_wb', ngram_range= (5,5), stop_words = ENGLISH_STOP_WORDS)
    noun_vec = CountVectorizer(analyzer = 'char_wb')
    pos_vec = CountVectorizer()
    vec_tar = LabelEncoder()
    targets = df['Category']
        
    counts = count_vec.fit_transform(df['TokensNouns'])
    #print("Counts shape: " + str(counts.shape))
    nouns = noun_vec.fit_transform(df['Nouns'])
    #print("Nouns shape: " + str(nouns.shape))
    pos = pos_vec.fit_transform(df['POS'])
    #print("POS shape: " + str(pos.shape))
    
    
    tfidf_transformer = TfidfTransformer()
    tfidf = tfidf_transformer.fit_transform(counts)
    
    
    test = count_vec.transform(test_df['TokensNouns'])
    test = tfidf_transformer.transform(test)
    test_nouns = noun_vec.transform(test_df['Nouns'])
    test_pos = pos_vec.transform(test_df['POS'])
    
    #To combine or not to combine
    nouns_array = nouns.toarray()
    pos_array = pos.toarray()
    counts_array = counts.toarray()
    #combined = np.hstack((nouns_array, pos_array))
    combined = np.hstack((nouns_array, pos_array))
    
    tnouns_array = test_nouns.toarray()
    tpos_array = test_pos.toarray()
    tcounts_array = test.toarray()
    
    #tcombined = np.hstack((tnouns_array, tpos_array))
    tcombined = np.hstack((tnouns_array, tpos_array))
    
    return tfidf, test

In [108]:
# Takes in training data, training labels, predicts labels for xtest using three methods.
def ensemble_predictor(df, test_df): 
    mod1 = LinearSVC(C=1000) #vec analyzer = word, ngram(1,1)
    mod2 = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
    mod3 = LogisticRegression(C=1000)#vec analyzer = word, ngram(1,3)
    mod4 = KNeighborsClassifier(n_neighbors=5)
    
    ytrain = df['Category']
    xtrain1, xtest1 = featurize(df, test_df, CountVectorizer(analyzer='word', ngram_range=(1,1)))
    xtrain2, xtest2 = featurize(df, test_df, CountVectorizer(analyzer='word', ngram_range=(1,1)))
    xtrain3, xtest3 = featurize(df, test_df, CountVectorizer(analyzer='word', ngram_range=(1,3)))
    xtrain4, xtest4 = featurize(df, test_df, CountVectorizer(analyzer='word', ngram_range=(1,3)))
    
    
    mod1.fit(xtrain1, ytrain)
    mod2.fit(xtrain2, ytrain)
    mod3.fit(xtrain3, ytrain)
    mod4.fit(xtrain4, ytrain)
    
    y1 = mod1.predict(xtest1)
    y2 = mod2.predict(xtest2)
    y3 = mod3.predict(xtest3)
    y4 = mod4.predict(xtest4)
    
    votes = zip(y1, y2, y3, y4)
    
    ypredicted = []
    count = 0
    for v in votes:
        v = list(v)
        shuffle(v)
        p = max(set(v), key=v.count)
        if len(np.unique(v)) == len(v):
            count += 1
        ypredicted.append(p)
    print("count: " + str(count))
    print("total number of predictions: " + str(len(ypredicted)))
    return ypredicted

In [100]:
#evaluate using 10 fold cross-validation, inspect results
#KNN WITH CV#
def cross_validate(targets, df):
    #SVC
    mod = SVC(C=1000)
    cv = StratifiedShuffleSplit(targets, n_iter=10, test_size=.1)
    
    scores = []
    for tr, tt in cv:
        train = df.loc[tr]
        test = df.loc[tt]
        result = featurize(train, test, CountVectorizer(analyzer='word', ngram_range=(1,1)))
        xtrain = result[0]
        xtest = result[1]
        mod.fit(xtrain, targets[tr])
        scores.append(mod.score(xtest, targets[tt]))

    print('\nLinear SVC\n\t mean score: {0}'.format(np.mean(scores)))
    
    
    #naive bayes
    mod = MultinomialNB(alpha=1.0, fit_prior=True)
    scores = []
    for tr, tt in cv:
        train = df.loc[tr]
        test = df.loc[tt]
        result = featurize(train, test, CountVectorizer(analyzer='word', ngram_range=(1,1)))
        xtrain = result[0]
        xtest = result[1]
        mod.fit(xtrain, targets[tr])
        scores.append(mod.score(xtest, targets[tt]))
    print('\nNaive Bayes\n\t mean score: {0}'.format(np.mean(scores)))
    
    
    mod = LogisticRegression(C=1000)
    scores = []
    for tr, tt in cv:
        train = df.loc[tr]
        test = df.loc[tt]
        result = featurize(train, test, CountVectorizer(analyzer='word', ngram_range=(1,3)))
        xtrain = result[0]
        xtest = result[1]
        mod.fit(xtrain, targets[tr])
        scores.append(mod.score(xtest, targets[tt]))
    print('\nLogReg\n\t mean score: {0}'.format(np.mean(scores)))
    
    
    mod = KNeighborsClassifier(n_neighbors=5)
    scores = []
    for tr, tt in cv:
        train = df.loc[tr]
        test = df.loc[tt]
        result = featurize(train, test, CountVectorizer(analyzer='word', ngram_range=(1,3)))
        xtrain = result[0]
        xtest = result[1]
        mod.fit(xtrain, targets[tr])
        scores.append(mod.score(xtest, targets[tt]))
    print('\nKNN\n\t mean score: {0}'.format(np.mean(scores)))
    
    
    
    scores = []
    for tr, tt in cv:
        train = df.loc[tr]
        test = df.loc[tt]
        predictions = ensemble_predictor(train, test)
        scores.append(accuracy_score(targets[tt], predictions))
    print('\nEnsemble Predictor\n\t mean score: {0}'.format(np.mean(scores)))
    


In [63]:
def plot_confusion_matrix(cm, title, target_names, cmap=plt.cm.coolwarm):
    plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [64]:
def examine_classification(model):
    result = get_labels_features(df, 1000)
    y = result[0]
    x = result[1]
    xtrain, xtest, ytrain, ytest = train_test_split(x, labels, test_size=0.33, random_state=99)
    
    model.fit(xtrain, ytrain)
    ypredicted = model.predict(xtest)
    print(accuracy_score(ytest, ypredicted))
    print()
    print(pd.crosstab(ytest, ypredicted, 
            rownames=['True'], colnames=['Predicted'], 
            margins=True))
    print()
    cm = confusion_matrix(ytest, ypredicted)
    plot_confusion_matrix(cm, "Confusion Matrix", range(1,8))

In [65]:
df = get_data()
df = add_features(df)
df.head()

,Category,Text,Tokens,Nouns,POS,TokenString,TokensNouns
0,5,why are yawns contagious? when people yawn,"[why, are, yawns, contagious, when, people, yawn]",yawn people,WRB VBP NNS JJ WRB NNS VBP,why are yawns contagious when people yawn,why are yawns contagious when people yawn yawn...
1,6,what is trans fat? how to reduce that? i heard...,"[what, is, trans, fat, how, to, reduce, that, ...",trans tras body food,WP VBZ NNS JJ WRB TO VB IN PRP VBP IN NNS VBP ...,what is trans fat how to reduce that i heard t...,what is trans fat how to reduce that i heard t...
2,1,roth ira vs 401k? what is the difference betwe...,"[roth, ira, vs, 401k, what, is, the, differenc...",roth ira v difference roth ira prefer,NN NN NNS CD WP VBZ DT NN IN NN NN CC CD WRB M...,roth ira vs 401k what is the difference betwee...,roth ira vs 401k what is the difference betwee...
3,1,how many planes fedex has? i heard that it is ...,"[how, many, planes, fedex, has, i, heard, that...",plane fedex airline world,WRB JJ NNS NN VBZ PRP VBP IN PRP VBZ DT JJS NN...,how many planes fedex has i heard that it is t...,how many planes fedex has i heard that it is t...
4,2,what is the best photo slideshow creation appl...,"[what, is, the, best, photo, slideshow, creati...",photo slideshow creation application photo sli...,WP VBZ DT JJS NN NN NN NN WP VBZ DT JJS NN NN ...,what is the best photo slideshow creation appl...,what is the best photo slideshow creation appl...


In [109]:
cross_validate(df['Category'], df)


Linear SVC
	 mean score: 0.44185185185185183

Naive Bayes
	 mean score: 0.42037037037037034

LogReg
	 mean score: 0.5662962962962962

KNN
	 mean score: 0.4614814814814815
count: 1
total number of predictions: 270
count: 3
total number of predictions: 270
count: 3
total number of predictions: 270
count: 6
total number of predictions: 270
count: 2
total number of predictions: 270
count: 2
total number of predictions: 270
count: 4
total number of predictions: 270
count: 4
total number of predictions: 270
count: 2
total number of predictions: 270
count: 2
total number of predictions: 270

Ensemble Predictor
	 mean score: 0.5211111111111111


In [84]:
test_df = get_test_data()
test_df = add_features(test_df)

In [110]:
predictions = ensemble_predictor(df, test_df)

count: 40
total number of predictions: 1874


In [111]:
import csv
with open('submission4.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['ID', 'Category'])
    for i, p in enumerate(predictions):
        writer.writerow([i + 1, p])